<a href="https://colab.research.google.com/github/datastax/ragstack-ai/blob/main/examples/notebooks/quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q ragstack-ai datasets

In [2]:
import cassio

In [3]:
import os
from getpass import getpass

# Enter your settings for Astra DB and OpenAI:
os.environ["ASTRA_DB_API_ENDPOINT"] ="enter astra db api endpoint"
os.environ["ASTRA_DB_APPLICATION_TOKEN"] ="enter astra db application token"
os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API Key: ")

## Create RAG Pipeline

### Embedding Model and Vector Store

In [5]:
pip install -U langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 317.4 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: langchain-openai
    Found existing installation: langchain-openai 0.1.8
    Uninstalling langchain-openai-0.1.8:
      Successfully uninstalled langchain-openai-0.1.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ragstack-ai-langchain 1.1.0 requires langchain-openai==0.1.8, but you have langchain-openai 0.1.13 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_astradb import AstraDBVectorStore
from langchain_openai import OpenAIEmbeddings
import os

# Configuring my embedding model and vector store
embedding = OpenAIEmbeddings()
vstore = AstraDBVectorStore(
    collection_name="test",
    embedding=embedding,
    token=os.getenv("ASTRA_DB_APPLICATION_TOKEN"),
    api_endpoint=os.getenv("ASTRA_DB_API_ENDPOINT"),
)
print("Astra vector store configured")

Astra vector store configured


In [5]:
from datasets import load_dataset

# Load a sample dataset
philo_dataset = load_dataset("datastax/philosopher-quotes")["train"]
print("An example entry:")
print(philo_dataset[16])

/opt/homebrew/Caskroom/miniconda/base/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 450/450 [00:00<00:00, 22942.82 examples/s]

An example entry:
{'author': 'aristotle', 'quote': 'Love well, be loved and do something of value.', 'tags': 'love;ethics'}


In [6]:
from langchain.schema import Document

# Constructs a set of documents from your data. Documents can be used as inputs to your vector store.
docs = []
for entry in philo_dataset:
    metadata = {"author": entry["author"]}
    if entry["tags"]:
        # Add metadata tags to the metadata dictionary
        for tag in entry["tags"].split(";"):
            metadata[tag] = "y"
    # Create a LangChain document with the quote and metadata tags
    doc = Document(page_content=entry["quote"], metadata=metadata)
    docs.append(doc)

In [7]:
docs

[Document(page_content="True happiness comes from gaining insight and growing into your best possible self. Otherwise all you're having is immediate gratification pleasure, which is fleeting and doesn't grow you as a person.", metadata={'author': 'aristotle', 'knowledge': 'y'}),
 Document(page_content='The roots of education are bitter, but the fruit is sweet.', metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}),
 Document(page_content='Before you heal the body you must first heal the mind', metadata={'author': 'aristotle', 'ethics': 'y'}),
 Document(page_content='The proof that you know something is that you are able to teach it', metadata={'author': 'aristotle', 'education': 'y', 'knowledge': 'y'}),
 Document(page_content='Those who are not angry at the things they should be angry at are thought to be fools, and so are those who are not angry in the right way, at the right time, or with the right persons.', metadata={'author': 'aristotle'}),
 Document(page_content=

In [8]:
# Create embeddings by inserting your documents into the vector store.
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 450 documents.


In [9]:
# Checks your collection to verify the documents are embedded.
print(vstore.astra_db.collection("test").find())

{'data': {'documents': [{'_id': 'febd28daa7744273a8a7d339288fa5e0', 'content': 'Perhaps the gods are kind to us, by making life more disagreeable as we grow older. In the end death seems less intolerable than the manifold burdens we carry', 'metadata': {'author': 'freud', 'religion': 'y'}}, {'_id': '9523b4b731df420e9fc16e879d48c202', 'content': 'It is the business of the very few to be independent; it is a privilege of the strong.', 'metadata': {'author': 'nietzsche', 'ethics': 'y', 'knowledge': 'y', 'politics': 'y'}}, {'_id': '8a129ae1a9334e4d86e383ec27a51244', 'content': 'Philosophy is by its nature something esoteric, neither made for the mob nor capable of being prepared for the mob.', 'metadata': {'author': 'hegel'}}, {'_id': '702f12bd92d94525b36cbab74eb49c92', 'content': 'When a man is prey to his emotions, he is not his own master.', 'metadata': {'author': 'spinoza', 'ethics': 'y'}}, {'_id': 'b3f904aa40b240be95727ce9b669dcd9', 'content': 'The secret of reaping the greatest fruit

### Basic Retrieval

Retrieve context from your vector database, and pass it to the model with a prompt.

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retriever = vstore.as_retriever(search_kwargs={"k": 3})

prompt_template = """
Answer the question based only on the supplied context. If you don't know the answer, say you don't know the answer.
Context: {context}
Question: {question}
Your answer:
"""
prompt = ChatPromptTemplate.from_template(prompt_template)
model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke("In the given context, what is happiness and can you explain more and who is the author?")

'In the given context, happiness is described as the reward of virtue. The author of this statement is Aristotle.'

In [ ]:
# Add your questions here!
# chain.invoke("<your question>")

## Cleanup

In [ ]:
# WARNING: This will delete the collection and all documents in the collection
vstore.delete_collection()

You now have a fully functioning RAG pipeline! Note that there are several different ways to accomplish this, depending on your input data format, vector store, embedding, model, output type, and more. There are also more advanced RAG techniques that leverage new ingestion, retrieval, and generation patterns.  

RAG is a powerful solution used in tandem with the capabilities of LLMs. Check out our other examples for ideas on how you can build innovative solutions using RAGStack!